In [5]:
import pandas as pd
import os
import datetime

In [6]:
def open_file(temp_path):
    if '.csv' in temp_path:
        return pd.read_csv(temp_path)
    if '.xlsx' in temp_path:
        return pd.read_excel(temp_path)

## 날짜 유혀성 확인
def validate(date_text):
    try:
        datetime.datetime.strptime(date_text, '%Y-%m-%d')
        return True
    except ValueError:
        return False

In [14]:
## 온도 데이터 위치
column_name = 'temp'
root_path = 'additional/temp'
save_path = 'temp_total.csv'
assert os.path.isdir(root_path), "온도 데이터가 있는 폴더 설정 하세요."

In [15]:
## 데이터 확인
filelist = [item for item in os.listdir(root_path) if '.csv' in item or '.xlsx' in item]
print(filelist)

['temp_2018.xlsx', 'temp_2015.xlsx', 'temp_2017.xlsx', 'temp_2019.xlsx', 'temp_2016.xlsx', 'temp_2020.xlsx']


In [16]:
merge_dict = {
    'date':[],
    'val':[],
}

for filename in filelist:
    temp_year = filename.split('.')[0].split('_')[1]
    
    ## 데이터 하나씩 불러오기
    df = open_file(os.path.join(root_path, filename))
    
    ## 컬럼 바꾸기
    df.columns = ['일', '01', '02', '03', '04', '05', '06', '07', '08', '09','10', '11', '12']
    df['일']=[str(idx+1).zfill(2) for idx in range(31)]
    
    month_columns = ['01', '02', '03', '04', '05', '06', '07', '08', '09','10', '11', '12']
    for temp_month in month_columns:
        temp_df = df[['일', temp_month]]
        
        for item in temp_df.values.tolist():
            temp_day = item[0]
            temp_data = item[1]
            
            temp_date = "{}-{}-{}".format(temp_year, temp_month, temp_day)
            
            ## 날짜 형식 확인
            if validate(temp_date):
                merge_dict['date'].append(temp_date)
                merge_dict['val'].append(temp_data)
                

In [17]:
merge_df = pd.DataFrame(merge_dict)

In [18]:
merge_df = merge_df.sort_values('date')
merge_df = merge_df.reset_index(drop=True)

In [19]:
merge_df.columns = ['date']+[column_name]

In [20]:
## 저장하기
merge_df.to_csv(save_path, index=None)